In [365]:
import numpy as np
import cv2
from skimage import io
from matplotlib import pyplot as plt
from collections import defaultdict
import SimpleITK as sitk#读取医疗图像的包
import numba as nb
BPLINE_BOARD_SIZE=3

In [366]:
#FFD代码:
@nb.jit()
def Bspline_Ffd_kernel(source, out, row_block_num, col_block_num, grid_points):
    row, col = source.shape
    delta_x = col*1.0/col_block_num #列的块长度
    delta_y = row*1.0/row_block_num#行的块长度
    grid_rows = row_block_num + BPLINE_BOARD_SIZE
    grid_cols = col_block_num + BPLINE_BOARD_SIZE #加padding目的是防止越界
    grid_size = grid_rows*grid_cols #网格大小
    for y in range(row): 
      for x in range(col):
        x_block = x / delta_x #在第几块
        y_block = y / delta_y #在第几块
        
        j = int(x_block)#取整
        i = int(y_block)#取整
        u = x_block - j#该点距离块边界的距离
        v = y_block - i

        #B样条基函数
        pX=np.zeros((4))
        pY=np.zeros((4))
        u2 = u*u
        u3 = u2*u
        pX[0] = (1 - u3 + 3 * u2 - 3 * u) / 6.0
        pX[1] = (4 + 3 * u3 - 6 * u2) / 6.0
        pX[2] = (1 - 3 * u3 + 3 * u2 + 3 * u) / 6.0
        pX[3] = u3 / 6.0
        v2 = v*v #v平方
        v3 = v2*v#v三次方
        pY[0] = (1 - v3 + 3 * v2 - 3 * v) / 6.0
        pY[1] = (4 + 3 * v3 - 6 * v2) / 6.0
        pY[2] = (1 - 3 * v3 + 3 * v2 + 3 * v) / 6.0
        pY[3] = v3 / 6.0

        Tx = x
        Ty = y
        for m in range(4):
          for n in range(4):
            control_point_x = j + n
            control_point_y = i + m #控制点的位置
            temp = pY[m] * pX[n] #B样条
            # print(control_point_y*grid_cols+control_point_x)
            Tx += temp*grid_points[int(control_point_y*grid_cols+control_point_x)]#累加x
            Ty += temp*grid_points[int(control_point_y*grid_cols+control_point_x+grid_size)]
        x1 = int(Tx)
        y1 = int(Ty)
        
        if (x1 < 1 or x1 >= col-1 or y1 < 1 or y1 >= row-1):
          out[y,x] = 0    #直接设为黑色
        else:
          x2 = x1 + 1   
          y2 = y1 + 1
          # print(y1,col,x1)
          #双线性插值
          gray = (x2 - Tx)*(y2 - Ty)*source[y1,x1] - \
              (x1 - Tx)*(y2 - Ty)*source[y1,x2] - (x2 - Tx)*(y1 - Ty)*source[y2,x1] + \
                              (x1 - Tx)*(y1 - Ty)*source[y2,x2]
          out[y,x] = gray
    return out

In [367]:
def Bspline_Ffd_local(source, out, row_block_num, col_block_num, grid_points, x1, y1, x2, y2):
    '''
    改进的局部点变换代码
    '''
    row, col = source.shape
    delta_x = col*1.0/col_block_num #列的块长度
    delta_y = row*1.0/row_block_num#行的块长度
    grid_rows = row_block_num + BPLINE_BOARD_SIZE
    grid_cols = col_block_num + BPLINE_BOARD_SIZE #加padding目的是防止越界
    grid_size = grid_rows*grid_cols #网格大小

    # p_grid = grid_points[0];
    for y in range(y1, y2+1):
      for x in range(x1, x2+1):
        x_block = x / delta_x #在第几块
        y_block = y / delta_y #在第几块
        
        j = np.floor(x_block)#取整
        i = np.floor(y_block)#取整
        u = x_block - j#该点距离块边界的距离
        v = y_block - i

        #B样条基函数
        pX=np.zeros((4))
        pY=np.zeros((4))
        u2 = u*u
        u3 = u2*u
        pX[0] = (1 - u3 + 3 * u2 - 3 * u) / 6.0
        pX[1] = (4 + 3 * u3 - 6 * u2) / 6.0
        pX[2] = (1 - 3 * u3 + 3 * u2 + 3 * u) / 6.0
        pX[3] = u3 / 6.0
        v2 = v*v #v平方
        v3 = v2*v#v三次方
        pY[0] = (1 - v3 + 3 * v2 - 3 * v) / 6.0
        pY[1] = (4 + 3 * v3 - 6 * v2) / 6.0
        pY[2] = (1 - 3 * v3 + 3 * v2 + 3 * v) / 6.0
        pY[3] = v3 / 6.0

        Tx = x
        Ty = y
        for m in range(4):
          for n in range(4):
            control_point_x = j + n
            control_point_y = i + m #控制点的位置
            temp = pY[m] * pX[n] #B样条
            Tx += temp*grid_points[int(control_point_y*grid_cols+control_point_x)]#累加x
            Ty += temp*grid_points[int(control_point_y*grid_cols+control_point_x+grid_size)]
        x1 = np.int64(Tx)
        y1 = np.int64(Ty)
        
        if (x1 < 1 or x1 >= col-1 or y1 < 1 or y1 >= row-1):
          out[y,x] = 0    #直接设为黑色
        else:
          x2 = x1 + 1   
          y2 = y1 + 1
          #双线性插值
          gray = (x2 - Tx)*(y2 - Ty)*source[y1,x1] - \
              (x1 - Tx)*(y2 - Ty)*source[y1,x2] - (x2 - Tx)*(y1 - Ty)*source[y2,x1] + \
                              (x1 - Tx)*(y1 - Ty)*source[y2,x2]
          out[y,x] = gray
    return out

In [368]:
# 随机生成2*(row_block_num+3)*(col_block_num+3)个范围在min~max之间的随机数作为控制参数：
def init_param(source, row_block_num, col_block_num, min_x, max_x):
  grid_rows = row_block_num + BPLINE_BOARD_SIZE
  grid_cols = col_block_num + BPLINE_BOARD_SIZE#网格的行和列
  grid_size = grid_rows*grid_cols
  grid_points = (max_x-min_x) * np.random.random_sample((2*grid_size)) +min_x #生成一维向量，保存x和y，用随机数初始化
  return grid_points

In [369]:
# '''
# FFD代码效果测试
# '''
# source = cv2.imread("float2.jpg" ,cv2.IMREAD_GRAYSCALE)   
# target = cv2.imread("base2.jpg" ,cv2.IMREAD_GRAYSCALE)    
# target = cv2.resize(target, source.shape[::-1])
# row_block_num = 30
# col_block_num = 30
# grid_points = init_param(target, row_block_num, col_block_num, -10, 10)
# out = target.copy()
# out = Bspline_Ffd_kernel(target, out, row_block_num, col_block_num, grid_points)
# plt.subplot(1, 2, 1), plt.axis("off")
# plt.imshow(target,cmap='gray'),plt.title('baseline')
# plt.subplot(1, 2, 2), plt.axis("off")
# plt.imshow(out,cmap='gray'),plt.title('out')

In [370]:
# # FFD LOCAL代码效果测试
# img = cv2.imread("base2.jpg", cv2.IMREAD_GRAYSCALE)
# row_block_num = 3
# col_block_num = 3
# grid_points = init_param(img, row_block_num, col_block_num,-10, 10);
# out = img.copy()
# out1 = Bspline_Ffd_kernel(img, out, row_block_num, col_block_num, grid_points)

# pos = 28
# grid_points[pos] += 0.5;

# # diff = abs(out1- out2)
# row, col = img.shape
# out2 = Bspline_Ffd_kernel(img, out, row_block_num, col_block_num, grid_points)
# x1,y1,x2,y2 = cal_rect_area(pos, row, col, row_block_num, col_block_num)

# # draw_1=cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
# out3 = Bspline_Ffd_local(img, out,row_block_num,col_block_num,grid_points,x1,y1,x2,y2)
# cv2.imwrite('2-3.jpg',abs(out2-out3))
# cv2.imwrite('1-3.jpg',abs(out1-out3))
# # print(out1-out3)

# plt.subplot(2, 2, 1), plt.axis("off")
# plt.imshow(img,cmap='gray'),plt.title('base')
# plt.subplot(2, 2, 2), plt.axis("off")
# plt.imshow(out1,cmap='gray'),plt.title('200')
# plt.subplot(2, 2, 3), plt.axis("off")
# plt.imshow(out2,cmap='gray'),plt.title('350')
# cv2.imwrite('out.jpg', out) 
# plt.subplot(2, 2, 4), plt.axis("off")
# plt.imshow(out3,cmap='gray'),plt.title('local 350')


In [371]:
# @nb.jit()
def cal_ncc(source, target, row, col):
    '''
    归一化互相关系数，用分块计算的方法
    input: source, target, row, col
    output: ncc
    '''
    row_size = int(source.shape[0]/row)
    col_size = int(source.shape[1]/col)
    ncc=0
    for i in range(row):
        i_begin = i*row_size
        for j in range(col):
            sum1=0
            sum2=0
            sum3=0
            j_begin = j*col_size
            for t1 in range(i_begin,np.min((source.shape[0],i_begin+row_size))):
                for t2 in range(j_begin,np.min((source.shape[1],j_begin+col_size))):
                    sum1+=int(source[t1,t2])*int(target[t1,t2])#a*b,转成int防止数值溢出
                    sum2+=np.square(source[t1,t2],dtype='int64') #a**2，转成int64防止数值溢出
                    sum3+=np.square(target[t1,t2],dtype='int64') #b**2，转成int64防止数值溢出
            # print(sum1)
            ncc+=np.sqrt(sum2*sum3)/(sum1+0.0000000001)#防止分母为0
    ncc/=(row*col)
    return ncc


#SSD相似性度量
@nb.jit()
def SSD_similarity(source,target):
    '''
    SSD相似性度量
    :param source， target 浮动图像和基准图像
    :output SSD score
    '''
    return ((source-target)**2).mean()

In [372]:
def get_fun_bpline(source, target, grid_points, row_block_num, col_block_num):
    '''
    计算目标函数 包含三部分: transform+插值+计算互相关
    input: source, target, grid_points
    output: ncc value
    '''
    # U,U_target = np.array_split(grid_points, 2)
    # U = U.reshape((-1,2))
    # U_target = U_target.reshape((-1,2))#提取出两个控制点
    out = np.zeros_like(source)
    out = Bspline_Ffd_kernel(source, out,row_block_num, col_block_num, grid_points)# 得到Y和X对应关系
    result = cal_ncc(out,target,5,5)#分5*5的小块计算互相关
    # result = SSD_similarity(out,target)
    # print('result!',result)
    return result

In [373]:
def cal_gradient(source,target,grid_points,gradient,row_block_num,col_block_num):
    '''
    计算梯度
    input: source, target,grid_points, old gradient
    output: new gradient update
    '''
    eps=1#超参数，可以改
    a1=get_fun_bpline(source,target,grid_points,row_block_num,col_block_num)
    grid_p = grid_points.copy()
    for i in range(grid_points.shape[0]):
        grid_p[i]+=eps #修改grid的i位元素的大小
        a2 = get_fun_bpline(source, target, grid_p,row_block_num, col_block_num) #每次都要做一次仿射+相似度估计
        grid_p[i]-=eps #再把它改回来
        # print(a2,a1)
        gradient[i] = (a2-a1)/eps#梯度
        # print('第',i,'个梯度大小为：',gradient[i])
    return gradient

In [374]:
def cal_rect_area(pos, row, col, row_block_num, col_block_num):
  '''
  判断控制点处于哪个block内
  '''
  grid_cols = col_block_num + BPLINE_BOARD_SIZE
  delta_x = col*1.0 / col_block_num
  delta_y = row*1.0 / row_block_num

  control_point_xx = pos % grid_cols
  control_point_yy = pos / grid_cols
  x1 = np.floor((control_point_xx - 3)*delta_x)
  y1 = np.floor((control_point_yy - 3)*delta_y)
  x2 = np.floor((control_point_xx + 1)*delta_x)
  y2 = np.floor((control_point_yy + 1)*delta_y)
  x1 = 0 if (x1 <= 0) else x1
  y1 = 0 if (y1 <= 0) else y1
  x2 = col-1 if (x2 > col - 1) else x2
  y2 = row-1 if (y2 > row - 1) else y2
  return int(x1),int(y1),int(x2),int(y2)

#快速计算总体的cc，涉及到十位数的乘法和开跟，会占用更多空间
def cal_ncc1(source,a1):
  sum1=0
  sum2=0
  sum3=0
  row,col = source.shape
  for i in range(row):
    for j in range(col):
      sum1 += np.int64(source[i,j]) * np.int64(source[i,j])#变量改成int64，因为十位数的乘法会超出范围
      sum2 += np.int64(source[i,j]) * np.int64(a1[i,j])
      sum3 += np.int64(a1[i,j]) * np.int64(a1[i,j])
  # 计算NCC的倒数作为目标函数值
  ncc1 = np.sqrt(sum1*sum3) / (sum2 + 0.0000001)

  return ncc1,sum1,sum2,sum3

#快速计算局部点的ncc
def cal_ncc2(source, a1, a2, x1,x2,y1,y2, sum1, sum2, sum3):
  sum2_i = sum2.copy()
  sum3_i = sum3.copy()
  for i in range(y1,y2+1):
    for j in range(x1,x2+1):
      # sum1没变
      sum2_i = sum2_i -int(source[i,j])*int(a1[i,j]) + int(source[i,j]) *int(a2[i,j])#sum2扣掉a1有关的，加上和a2有关的
      sum3_i = sum3_i - int(a1[i,j]) * int(a1[i,j]) + int(a2[i,j]) * int(a2[i,j])#sum3扣掉a1有关的，加上和a2有关的
  #计算NCC的倒数作为目标函数值
  cc2 = np.sqrt(sum1*sum3_i) / (sum2_i + 0.0000001)
  return cc2

def cal_gradient_local(source,target,grid_points,gradient,row_block_num,col_block_num):
  '''
  局部FFD方法计算梯度
  '''
  eps = 5
  row, col = source.shape
  grid_size = np.int64(grid_points.shape[0] / 2)
  out = source.copy()
  a1 = Bspline_Ffd_kernel(source, out, row_block_num, col_block_num, grid_points)
  cc1,sum1,sum2,sum3= cal_ncc1(target,a1)
  grid_p = grid_points.copy()
  for i in range(grid_size):
    grid_p[i] += eps
    x1,y1,x2,y2 = cal_rect_area(i, row, col, row_block_num, col_block_num)
    a2 = Bspline_Ffd_local(source, out, row_block_num, col_block_num, grid_p, x1, y1, x2, y2)
    cc2 = cal_ncc2(target,a1,a2,x1,x2,y1,y2,sum1,sum2,sum3)
    grid_p[i] -= eps
    gradient[i] = (cc2 - cc1) / eps

    grid_p[i + grid_size] += eps
    a2 = Bspline_Ffd_local(source, out, row_block_num, col_block_num, grid_p, x1, y1, x2, y2)
    cc2 = cal_ncc2(target,a1,a2,x1,x2,y1,y2,sum1,sum2,sum3)
    grid_p[i + grid_size] -= eps
    gradient[i + grid_size] = (cc2 - cc1) / eps
  return gradient

In [375]:
def update_grid_points(grid_points, gradient, alpha):
    '''
    根据梯度下降的逻辑更新grid
    '''
    for i in range(grid_points.shape[0]):
        grid_points[i]=grid_points[i]-gradient[i]*alpha
    return grid_points

In [376]:
def optimization_gd(source, target, out, row_block_num, col_block_num,grid_points):
    '''
    最优化迭代的具体步骤
    超参数需要进一步调整
    '''
    max_iter=500#最多迭代次数
    e =0.0001#迭代精度
    count=0#迭代次数统计
    alpha = 800000#初始alpha
    gradient = np.zeros_like(grid_points,dtype='float')#初始化梯度
    gradient = cal_gradient_local(source, target, grid_points, gradient, row_block_num,col_block_num)
    # print(gradient)
    out_cnt=0
    while count<max_iter:
        pred_grid_points= grid_points.copy()
        grid_points = update_grid_points(grid_points,gradient,alpha)
        ret1 = get_fun_bpline(source, target, pred_grid_points,row_block_num,col_block_num)#原本点的相关系数
        ret2 = get_fun_bpline(source, target, grid_points,row_block_num,col_block_num)#更新后点的相关系数
        print('original:%9f'%ret1,'after: %9f'%ret2,', alpha=',alpha)
        if ret2>ret1: #如果当前轮迭代的目标函数值大于上一轮的函数值，则减小步长并重新计算梯度、重新更新参数
            if alpha < 0.0001: 
                '''这一块到时候可能要删，因为现在跑的时候alpha都1e-6了还在减小'''
                float = Bspline_Ffd_kernel(target, out, row_block_num, col_block_num, grid_points)# 得到Y和X对应关系
                return float
            alpha*=0.5
            grid_points = pred_grid_points.copy()
            print('update alpha...')
            continue
        if np.abs(ret2-ret1)<e: #如果前后的变化比e小
            out_cnt+=1
            print('small enough!')
            if out_cnt>=2: #如果连续2次目标函数值不变则认为达到最优解停止迭代
                float = Bspline_Ffd_kernel(target,  out, row_block_num, col_block_num, grid_points)# 得到Y和X对应关系
                return float
        else:
            out_cnt=0
        pre_gradient = gradient.copy()
        gradient = cal_gradient_local(source, target, grid_points,gradient,row_block_num,col_block_num)#计算新的梯度
        if np.linalg.norm(gradient,2)>np.linalg.norm(pre_gradient,2): #如果新的梯度比原来的二范数大，则增加alpha步长
            print('add alpha')
            alpha*=2
        count+=1
        print('第 ',count,'次迭代:')
    return -1

In [377]:
def optimization_gd_ffd(source, target, out, row_block_num, col_block_num,grid_points):
    '''
    最优化迭代的具体步骤
    超参数需要进一步调整
    '''
    max_iter=500#最多迭代次数
    e =0.0001#迭代精度
    count=0#迭代次数统计
    alpha = 5000#初始alpha
    gradient = np.zeros_like(grid_points,dtype='float')#初始化梯度
    gradient = cal_gradient(source, target, grid_points, gradient, row_block_num,col_block_num)
    out_cnt=0
    while count<max_iter:
        pred_grid_points= grid_points.copy()
        grid_points = update_grid_points(grid_points,gradient,alpha)
        ret1 = get_fun_bpline(source, target, pred_grid_points,row_block_num,col_block_num)#原本点的相关系数
        ret2 = get_fun_bpline(source, target, grid_points,row_block_num,col_block_num)#更新后点的相关系数
        print('original:%9f'%ret1,'after: %9f'%ret2,', alpha=',alpha)
        if ret2>ret1: #如果当前轮迭代的目标函数值大于上一轮的函数值，则减小步长并重新计算梯度、重新更新参数
            if alpha < 0.0001: 
                float = Bspline_Ffd_kernel(target, out, row_block_num, col_block_num, grid_points)# 得到Y和X对应关系
                print('alpha too small')
                return float
            alpha*=0.5
            grid_points = pred_grid_points.copy()
            print('update alpha...')
            continue
        if np.abs(ret2-ret1)<e: #如果前后的变化比e小
            out_cnt+=1
            print('small enough!')
            if out_cnt>=2: #如果连续2次目标函数值不变则认为达到最优解停止迭代
                float = Bspline_Ffd_kernel(target,  out, row_block_num, col_block_num, grid_points)# 得到Y和X对应关系
                return float
        else:
            out_cnt=0
        pre_gradient = gradient.copy()
        gradient = cal_gradient(source, target, grid_points,gradient,row_block_num,col_block_num)#计算新的梯度
        if np.linalg.norm(gradient,2)>np.linalg.norm(pre_gradient,2): #如果新的梯度比原来的二范数大，则增加alpha步长
            print('add alpha')
            alpha*=2
        count+=1
        print('第 ',count,'次迭代:')
    return -1

In [378]:
def oneDSearch(source, target,row_block_num,col_block_num,grid_points,d):
    '''
    输入：图片，特征点向量，一维搜索方向
    输出：新的grid point，最佳相似度
    '''
    max_iter=50#最多迭代次数
    e =0.0001#迭代精度
    count=0#迭代次数统计
    alpha = 80000#初始alpha
    a1=get_fun_bpline(source,target,grid_points,row_block_num,col_block_num)
    # temp = grid_points.copy()
    grid_p = grid_points.copy()
    grid_p+=d #修改grid的i位元素的大小
    a2 = get_fun_bpline(source, target, grid_p,row_block_num, col_block_num) #每次都要做一次仿射+相似度估计
    grid_p-=d
    # print(d)
    gradient = (a2-a1)/np.linalg.norm(d)#沿着d方向上的梯度
    out_cnt=0

    while count<max_iter:
        pred_grid_points= grid_points.copy()
        grid_points-=gradient*alpha*d
        ret1 = get_fun_bpline(source, target, pred_grid_points,row_block_num,col_block_num)#原本点的相关系数
        ret2 = get_fun_bpline(source, target, grid_points,row_block_num,col_block_num)#更新后点的相关系数
        # print('original:%9f'%ret1,'after: %9f'%ret2,', alpha=',alpha)
        if ret2>ret1: #如果当前轮迭代的目标函数值大于上一轮的函数值，则减小步长并重新计算梯度、重新更新参数
            alpha*=0.5
            grid_points = pred_grid_points
            continue
        if np.abs(ret2-ret1)<e: #如果前后的变化比e小
            out_cnt+=1
            return grid_points, ret2
        else:
            out_cnt=0
        pre_gradient = gradient.copy()
        a1=get_fun_bpline(source,target,grid_points,row_block_num,col_block_num)
        grid_p = grid_points.copy()
        grid_p+=d #修改grid的i位元素的大小
        a2 = get_fun_bpline(source, target, grid_p,row_block_num, col_block_num) #每次都要做一次仿射+相似度估计
        gradient = (a2-a1)/np.linalg.norm(d)#梯度
        if np.abs(gradient)>np.abs(pre_gradient): #如果新的梯度比原来的二范数大，则增加alpha步长
            # print('add alpha')
            alpha*=2
        count+=1
    return grid_points, 100
    



In [379]:
def optimization_powell(source, target, out, row_block_num, col_block_num,grid_points):
    '''
    powell算法
    输入： 图片，特征点
    输出： 配准结果图片
    '''
    e=0.01;
    x0 = grid_points.copy()
    num_para = len(x0)
    D=np.eye(num_para)
    count=0
    while True:
        fX0=get_fun_bpline(source, target, x0, row_block_num, col_block_num)
        xlist=np.array([])
        fxlist=np.array([])
        xi = x0.copy()
        for i in range(D.shape[0]):
            di = D[i]
            xi,fxi= oneDSearch(source, target,row_block_num,col_block_num,xi,di)
            print(i,fxi)
            xlist = np.append(xlist,xi)
            fxlist = np.append(fxlist,fxi)
            # print(xlist,fxlist)
        #构造新的搜索方向
        xlist = xlist.reshape((-1,72))
        dnew = xi-x0
        dnew = dnew/np.linalg.norm(dnew,2)
        count+=1
        print('第',count,'次迭代')
        #沿着新的搜索方向找极小值点
        xnew,fxnew= oneDSearch(source, target,row_block_num,col_block_num,xi,dnew)
        print('x0',x0)
        print('xnew',xnew)
        temp1 = xnew-x0
        if np.linalg.norm(temp1,2)<e or np.abs(fxnew-fX0)<=e*fxnew: #检验是否满足终止条件
            break
        diff = np.array([fX0-fxlist[0]])
        for i in range(fxlist.shape[0]-1):
            np.append(diff,fxlist[i]-fxlist[i+1])
        maxdiff = np.amax(diff) #找下降最大的点
        print('maxdiff',maxdiff)
        maxpos = np.argmax(diff)
        xn=2*xlist[-1]-x0 #确定映射点
        fxn = get_fun_bpline(source, target, xn, row_block_num, col_block_num)
        if fxn<fX0 and (fX0-2*fxlist[-1]+fxn)*(fX0-fxlist[-1]-maxdiff)<0.5*maxdiff*(fX0-fxn)**2:
            x0 = xnew #xn作为迭代的新的起点
            D[maxpos]=dnew #修改迭代方向
        else:
            if fxlist[-1]<fxn:
                x0 = xlist[-1]#更改出发点,方向不变
            else: 
                x0 = xn

    return Bspline_Ffd_kernel(source,out,row_block_num,col_block_num,xnew)



In [380]:
source = cv2.imread("float_3.jpg" ,cv2.IMREAD_GRAYSCALE)   #浮动图像
target = cv2.imread("base_3.jpg" ,cv2.IMREAD_GRAYSCALE) #基准图像
target = cv2.resize(target, source.shape[::-1])
row_block_num = 3
col_block_num = 3
grid_points = init_param(source, row_block_num, col_block_num, -10, 10)
'''
如果row和col的大小是3，那么grid_points的长度为6*6*2=72,梯度的长度为72
'''
out = np.zeros_like(source,dtype='float')
out = optimization_gd_ffd(source, target, out, row_block_num, col_block_num,grid_points)

diff = np.abs(np.array(target,dtype='int64')-np.array(out,dtype='int64'))

plt.subplot(2, 2, 1), plt.axis("off")
plt.imshow(source,cmap='gray'),plt.title('float')
plt.subplot(2, 2, 2), plt.axis("off")
plt.imshow(target,cmap='gray'),plt.title('baseline')
plt.subplot(2, 2, 3), plt.axis("off")
plt.imshow(out,cmap='gray'),plt.title('out')
cv2.imwrite('out.jpg', out) 
plt.subplot(2, 2, 4), plt.axis("off")
plt.imshow(diff,cmap='gray'),plt.title('out-base')

original: 1.760105 after:  1.534115 , alpha= 5000
add alpha
第  1 次迭代:
original: 1.534115 after:  1.967959 , alpha= 10000
update alpha...
original: 1.534115 after:  1.959757 , alpha= 5000.0
update alpha...
original: 1.534115 after:  1.379520 , alpha= 2500.0
第  2 次迭代:
original: 1.379520 after:  1.347259 , alpha= 2500.0
第  3 次迭代:
original: 1.347259 after:  1.322820 , alpha= 2500.0
第  4 次迭代:
original: 1.322820 after:  1.301619 , alpha= 2500.0
第  5 次迭代:
original: 1.301619 after:  1.283375 , alpha= 2500.0
第  6 次迭代:
original: 1.283375 after:  1.267368 , alpha= 2500.0
第  7 次迭代:
original: 1.267368 after:  1.252994 , alpha= 2500.0
第  8 次迭代:
original: 1.252994 after:  1.240839 , alpha= 2500.0
第  9 次迭代:
original: 1.240839 after:  1.229932 , alpha= 2500.0
第  10 次迭代:
original: 1.229932 after:  1.220122 , alpha= 2500.0
第  11 次迭代:
original: 1.220122 after:  1.211391 , alpha= 2500.0
第  12 次迭代:
original: 1.211391 after:  1.203882 , alpha= 2500.0
第  13 次迭代:
original: 1.203882 after:  1.197539 , alpha= 25

KeyboardInterrupt: 

In [ ]:
# dnew = xi-x0
# dnew = dnew/np.linalg.norm(dnew,2)
# #沿着新的搜索方向找极小值点

# # xnew,fxnew= oneDSearch(source, target,row_block_num,col_block_num,xi,dnew)
# out = Bspline_Ffd_kernel(source,out,row_block_num,col_block_num,xnew)
# diff = np.abs(np.array(target,dtype='int64')-np.array(out,dtype='int64'))


# plt.subplot(2, 2, 1), plt.axis("off")
# plt.imshow(source,cmap='gray'),plt.title('float')
# plt.subplot(2, 2, 2), plt.axis("off")
# plt.imshow(target,cmap='gray'),plt.title('baseline')
# plt.subplot(2, 2, 3), plt.axis("off")
# plt.imshow(out,cmap='gray'),plt.title('out')
# cv2.imwrite('out.jpg', out) 
# plt.subplot(2, 2, 4), plt.axis("off")
# plt.imshow(diff,cmap='gray'),plt.title('out-base')
